# Neural Network Regression with TensorFlow

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [3]:
file_path = r'C:\Users\CBS\Python_ML_2023\Exercises\Final Project\df_wine_clean.csv'
df = pd.read_csv(file_path)

In [4]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [5]:
df1 = df.drop(['description', 'designation',   
       'region_1',  'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'], axis=1)


In [6]:
# Assuming df is your original DataFrame
sample_size = int(len(df) * 1)  # 100% of the data
sample_df = df1.sample(n=sample_size, random_state=42)

In [7]:
sample_df.columns

Index(['country', 'points', 'price', 'province', 'taster_name'], dtype='object')

In [8]:
for column in sample_df.columns:
    unique_values_sum = sample_df[column].nunique()
    print(f"Sum of unique values in {column}: {unique_values_sum}")


Sum of unique values in country: 43
Sum of unique values in points: 21
Sum of unique values in price: 542
Sum of unique values in province: 423
Sum of unique values in taster_name: 20


In [9]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ['price']), # get all values between 0 and 1
    (OneHotEncoder(handle_unknown="ignore"), ['country','province', 'taster_name'])
)


In [10]:
# Create X & y
X = sample_df.drop(['points'], axis=1)
y = sample_df['points']

In [11]:

# Build our train and test sets (use random state to ensure same split as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScalar) and one hot encoding (OneHotEncoder)
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [13]:
X_train_normal[0]

<1x475 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [14]:
X_train_dense = X_train_normal.toarray()
X_test_dense=X_test_normal.toarray()

In [15]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10, 1 units)
wine_model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
wine_model_1.compile(loss=tf.keras.losses.mse,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])


In [239]:

# Fit the model for 200 epochs 
wine_model_1.fit(X_train_dense, y_train, epochs=200, verbose=1) 

Epoch 1/200
3250/3250 [==============================] - 3s 827us/step - loss: 217.2125 - mse: 217.2125
Epoch 2/200
3250/3250 [==============================] - 3s 826us/step - loss: 7.4024 - mse: 7.4024
Epoch 3/200
3250/3250 [==============================] - 3s 825us/step - loss: 7.4167 - mse: 7.4167
Epoch 4/200
3250/3250 [==============================] - 3s 842us/step - loss: 7.4032 - mse: 7.4032
Epoch 5/200
3250/3250 [==============================] - 3s 865us/step - loss: 7.3862 - mse: 7.3862
Epoch 6/200
3250/3250 [==============================] - 3s 832us/step - loss: 7.3555 - mse: 7.3555
Epoch 7/200
3250/3250 [==============================] - 3s 845us/step - loss: 7.3578 - mse: 7.3578
Epoch 8/200
3250/3250 [==============================] - 3s 839us/step - loss: 7.3382 - mse: 7.3382
Epoch 9/200
3250/3250 [==============================] - 3s 840us/step - loss: 7.2974 - mse: 7.2974
Epoch 10/200
3250/3250 [==============================] - 3s 838us/step - loss: 7.3306 - mse: 7.

In [240]:
wine_model_1.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 100)               47600     
                                                                 
 dense_46 (Dense)            (None, 10)                1010      
                                                                 
 dense_47 (Dense)            (None, 1)                 11        
                                                                 
Total params: 48,621
Trainable params: 48,621
Non-trainable params: 0
_________________________________________________________________


In [241]:
# Evaulate 3rd model
wine_model_1_loss, wine_model_1_mse = wine_model_1.evaluate(X_test_dense, y_test)

813/813 [==============================] - 1s 603us/step - loss: 7.1382 - mse: 7.1382


# model 2

In [242]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [243]:
df2 = df.drop([ 'description', 'designation','clean_text','title'], axis=1)

In [244]:
df2.columns

Index(['country', 'points', 'price', 'province', 'region_1', 'taster_name',
       'variety', 'winery', 'taster_weight_cat', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [245]:
# Assuming df is your original DataFrame
sample_size = int(len(df2) * 0.2)  # 20% of the data
sample_df = df2.sample(n=sample_size, random_state=42)

In [246]:
for column in sample_df.columns:
    unique_values_sum = sample_df[column].nunique()
    print(f"Sum of unique values in {column}: {unique_values_sum}")

Sum of unique values in country: 38
Sum of unique values in points: 21
Sum of unique values in price: 349
Sum of unique values in province: 316
Sum of unique values in region_1: 934
Sum of unique values in taster_name: 20
Sum of unique values in variety: 473
Sum of unique values in winery: 9344
Sum of unique values in taster_weight_cat: 4
Sum of unique values in extracted_words: 1153
Sum of unique values in extracted_digits: 57


In [247]:

# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ['price']), # get all values between 0 and 1
    (OneHotEncoder(handle_unknown="ignore"),['country', 'province', 'region_1', 'taster_name',
       'variety', 'winery', 'taster_weight_cat', 'extracted_words',
       'extracted_digits'])
)


In [248]:
# Create X & y
X = sample_df.drop(['points'], axis=1)
y = sample_df['points']

In [249]:
# Build our train and test sets (use random state to ensure same split as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScalar) and one hot encoding (OneHotEncoder)
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [250]:
X_train_dense = X_train_normal.toarray()
X_test_dense=X_test_normal.toarray()

In [251]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10, 1 units)
wine_model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
wine_model_1.compile(loss=tf.keras.losses.mse,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])


In [252]:
# Fit the model for 5 epochs 
wine_model_1.fit(X_train_dense, y_train, epochs=5, verbose=1) 

Epoch 1/5
650/650 [==============================] - 10s 15ms/step - loss: 775.6989 - mse: 775.6989
Epoch 2/5
650/650 [==============================] - 10s 15ms/step - loss: 6.6277 - mse: 6.6277
Epoch 3/5
650/650 [==============================] - 10s 15ms/step - loss: 4.6861 - mse: 4.6861
Epoch 4/5
650/650 [==============================] - 10s 15ms/step - loss: 4.3943 - mse: 4.3943
Epoch 5/5
650/650 [==============================] - 9s 15ms/step - loss: 4.5045 - mse: 4.5045


In [253]:
# Fit the model for 50 epochs 
wine_model_1.fit(X_train_dense, y_train, epochs=50, verbose=0) 

In [254]:
wine_model_1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 100)               1125500   
                                                                 
 dense_49 (Dense)            (None, 10)                1010      
                                                                 
 dense_50 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,126,521
Trainable params: 1,126,521
Non-trainable params: 0
_________________________________________________________________


In [255]:
# Evaulate 2rd model
wine_model_1_loss, wine_model_1_mse = wine_model_1.evaluate(X_test_dense, y_test)

163/163 [==============================] - 0s 1ms/step - loss: 7.6817 - mse: 7.6817


# model 3

In [181]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [182]:
df3 = df.drop([ 'description', 'designation','clean_text','title','winery','extracted_words'], axis=1)

In [183]:
# Assuming df is your original DataFrame
sample_size = int(len(df3) * 1)  # 100% of the data
sample_df = df3.sample(n=sample_size, random_state=42)

In [184]:
for column in sample_df.columns:
    unique_values_sum = sample_df[column].nunique()
    print(f"Sum of unique values in {column}: {unique_values_sum}")

Sum of unique values in country: 43
Sum of unique values in points: 21
Sum of unique values in price: 542
Sum of unique values in province: 423
Sum of unique values in region_1: 1230
Sum of unique values in taster_name: 20
Sum of unique values in variety: 708
Sum of unique values in taster_weight_cat: 4
Sum of unique values in extracted_digits: 87


In [185]:
# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ['price']), # get all values between 0 and 1
    (OneHotEncoder(handle_unknown="ignore"),['country', 'province', 'region_1', 'taster_name',
       'variety',  'taster_weight_cat',
       'extracted_digits'])
)


In [186]:
# Create X & y
X = sample_df.drop(['points'], axis=1)
y = sample_df['points']

In [188]:
# Build our train and test sets (use random state to ensure same split as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScalar) and one hot encoding (OneHotEncoder)
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [189]:
X_train_dense = X_train_normal.toarray()
X_test_dense=X_test_normal.toarray()

In [220]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10, 1 units)
wine_model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

# Compile the model
wine_model_1.compile(loss=tf.keras.losses.mse,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])


In [221]:
# Fit the model for 5 epochs 
wine_model_1.fit(X_train_dense, y_train, epochs=5, verbose=1) 

Epoch 1/5
3250/3250 [==============================] - 5s 1ms/step - loss: 156.9802 - mse: 156.9802
Epoch 2/5
3250/3250 [==============================] - 4s 1ms/step - loss: 6.9048 - mse: 6.9048
Epoch 3/5
3250/3250 [==============================] - 4s 1ms/step - loss: 6.8139 - mse: 6.8139
Epoch 4/5
3250/3250 [==============================] - 4s 1ms/step - loss: 6.6546 - mse: 6.6546
Epoch 5/5
3250/3250 [==============================] - 4s 1ms/step - loss: 6.5789 - mse: 6.5789


In [222]:
# Evaulate 3rd model
wine_model_1_loss, wine_model_1_mse = wine_model_1.evaluate(X_test_dense, y_test)

813/813 [==============================] - 1s 699us/step - loss: 6.7001 - mse: 6.7001


In [223]:
# Fit the model for 50 epochs 
wine_model_1.fit(X_train_dense, y_train, epochs=50, verbose=1) 

Epoch 1/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.4911 - mse: 6.4911
Epoch 2/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.4699 - mse: 6.4699
Epoch 3/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.4141 - mse: 6.4141
Epoch 4/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.3515 - mse: 6.3515
Epoch 5/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.3402 - mse: 6.3402
Epoch 6/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.3031 - mse: 6.3031
Epoch 7/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.2877 - mse: 6.2877
Epoch 8/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.2765 - mse: 6.2765
Epoch 9/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.2535 - mse: 6.2535
Epoch 10/50
3250/3250 [==============================] - 4s 1ms/step - loss: 6.2625 - mse: 6.2625
Epoch 11/50
3250/3250 [======

In [224]:
# Evaulate 3rd model
wine_model_1_loss, wine_model_1_mse = wine_model_1.evaluate(X_test_dense, y_test)

813/813 [==============================] - 1s 761us/step - loss: 6.0244 - mse: 6.0244


### nlp 

In [12]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [13]:
# Shuffle training dataframe
train_df_shuffled = df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,taster_weight_cat,clean_text,extracted_words,extracted_digits
72773,France,This Chardonnay's acidity and minerally struct...,other,88,75.000000,Burgundy,Puligny-Montrachet,Roger Voss,Joseph Faiveley 2010 Puligny-Montrachet,Chardonnay,Joseph Faiveley,Category B,chardonnay acid miner structur fine balanc pal...,[],2010
90612,Italy,This blend of Garganega (the grape that makes ...,other,83,27.047835,Northeastern Italy,Delle Venezie,other,Duca del Frassino 2011 White (Delle Venezie),White Blend,Duca del Frassino,Category B,blend garganega grape make soav wine pinot gri...,['Delle Venezie'],2011
121327,France,"A concentrated, rich wine, packed with peach a...",Grand'Chaille,91,25.998123,Loire Valley,Sancerre,Roger Voss,Domaine Thomas & Fils 2008 Grand'Chaille (San...,Sauvignon Blanc,Domaine Thomas & Fils,Category B,concentr rich wine pack peach pear flavor fres...,['Sancerre'],2008
46544,US,"Lots of rich blueberry, cherry and blackberry ...",Hope Family Vineyard,87,42.000000,California,Paso Robles,other,Austin Hope 2011 Hope Family Vineyard Syrah (P...,Syrah,Austin Hope,Category B,lot rich blueberri cherri blackberri fruit fla...,['Paso Robles'],2011
186,US,This is a very rich Pinot whose primary virtue...,Wiley Vineyard,88,40.000000,California,Anderson Valley,other,Harrington 2006 Wiley Vineyard Pinot Noir (And...,Pinot Noir,Harrington,Category B,rich pinot whose primari virtu fruit explod de...,['Anderson Valley'],2006


In [14]:


# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["description"].to_numpy(),
                                                                            train_df_shuffled["points"].to_numpy(),
                                                                            test_size=0.2, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(103971, 103971, 25993, 25993)

In [16]:
### Text vectorization (tokenization)
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6


# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [17]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

40

In [18]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 40 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 40), dtype=int64, numpy=
array([[ 122,    4, 2370,   11, 4792, 7679,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0]], dtype=int64)>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
import random
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
From the former Ashley's Vineyard, one of the best-known in the AVA, comes this very ripe, extracted Pinot Noir. Tastes like it had a long hangtime, getting ripe to the point of baked pie-filling cherries and puréed, smoky, almost raisined blackberries. The tradeoff for all this richness is softness and a resulting one-dimensionality.      

Vectorized version:


<tf.Tensor: shape=(1, 40), dtype=int64, numpy=
array([[  22,    3, 3031,    1,  151,  157,    5,    3,    1,   11,    3,
        2097,  240,    7,   66,   27,  690,   98,  238,  203,  109,   12,
        1895,    4,  118, 3792, 2697,   27,   13,    3,  890,    5,  229,
        3137,  173,    2,    1,  169,  188, 5110]], dtype=int64)>

In [22]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'and', 'the', 'a']
Bottom 5 least common words: ['softbodied', 'soares', 'snacks', 'smidgens', 'smackingly']


In [ ]:
# Creating an Embedding using an Embedding Layer

In [23]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [24]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
This 100% varietal wine offers aromas of coffee, green herbs, pencil lead and vanilla followed by silky cherry and dark chocolate flavors. There's plenty of appeal but the variety seems to get lost.      

Embedded version:


<tf.Tensor: shape=(1, 40, 128), dtype=float32, numpy=
array([[[ 0.04853568, -0.04433579, -0.03145378, ...,  0.00175725,
          0.00621579,  0.04214795],
        [-0.04442856,  0.04505548,  0.02003416, ..., -0.00263672,
         -0.00598887, -0.0089368 ],
        [-0.04153187,  0.01386137, -0.0465541 , ...,  0.01098907,
         -0.01932958, -0.02483048],
        ...,
        [-0.01450722, -0.04854118,  0.02312665, ...,  0.02064779,
         -0.04664967,  0.03627522],
        [-0.01450722, -0.04854118,  0.02312665, ...,  0.02064779,
         -0.04664967,  0.03627522],
        [-0.01450722, -0.04854118,  0.02312665, ...,  0.02064779,
         -0.04664967,  0.03627522]]], dtype=float32)>

In [33]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03333165, -0.04857239, -0.01054839,  0.0272988 , -0.00382303,
       -0.0480469 ,  0.03834609, -0.02931286, -0.01784631,  0.00357044,
        0.04381588, -0.03250489,  0.02170971,  0.03339458, -0.03612515,
        0.03084313,  0.04319126,  0.00799993, -0.00022131,  0.02822665,
       -0.00847627,  0.03491584, -0.03364166, -0.0380974 ,  0.02328969,
       -0.03538864,  0.03966046, -0.02955651,  0.01785297,  0.01936351,
       -0.04831639, -0.0473676 ,  0.04827124, -0.00525031, -0.04869051,
        0.0066415 ,  0.02795322,  0.02386634, -0.01266098, -0.01881822,
        0.01192392, -0.04031923, -0.0169936 ,  0.00448623,  0.03942652,
       -0.01219716, -0.00602226, -0.02396879,  0.02290897,  0.00073887,
        0.00834299, -0.0428276 , -0.04159749, -0.04496205,  0.03248252,
        0.00365256,  0.04084495, -0.04093276, -0.04815407, -0.04578024,
       -0.03116703, -0.04572845, -0.02802062,  0.02249709, -0.01061878,
        0.034621

# Modelling a text dataset

# nlp- Model 0: Getting a baseline

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# Create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),  # convert words to numbers using tfidf
    ("regressor", LinearRegression())  # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('regressor', LinearRegression())])

In [35]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 63.29%


In [36]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([87.8161736 , 86.2279884 , 87.16783131, 92.42746795, 89.75495362,
       89.24717649, 83.01867574, 84.06951229, 89.59291174, 90.21629532,
       90.30972153, 95.41330441, 86.02482296, 91.58223797, 87.04616643,
       92.09174326, 86.79244805, 89.78592128, 87.92800359, 91.83048807])

In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_results(y_true, y_pred):
    """
    Calculates model MAE, MSE, and R-squared of a regression model.

    Args:
    -----
    y_true: true labels in the form of a 1D array
    y_pred: predicted labels in the form of a 1D array

    Returns a dictionary of MAE, MSE, and R-squared.
    """
    # Calculate MAE
    model_mae = mean_absolute_error(y_true, y_pred)
    # Calculate MSE
    model_mse = mean_squared_error(y_true, y_pred)
    # Calculate R-squared
    model_r2 = r2_score(y_true, y_pred)

    model_results = {"MAE": model_mae,
                     "MSE": model_mse,
                     "R-squared": model_r2}

    return model_results


In [38]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'MAE': 1.4076252944179053,
 'MSE': 3.4218086321893355,
 'R-squared': 0.6328651260319366}

# Model 1: A simple dense model

In [39]:
from tensorflow.keras import layers

# Define the model architecture
inputs = layers.Input(shape=(1,), dtype="string")  # Inputs are 1-dimensional strings
x = text_vectorizer(inputs)  # Turn the input text into numbers
x = embedding(x)  # Create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x)  # Lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="linear")(x)  # Create the output layer with linear activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")  # Construct the model



In [40]:
# Compile model for regression
model_1.compile(loss="mean_squared_error",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mean_squared_error"])


In [41]:
# Get a summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 40)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 40, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [298]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              )

Epoch 1/5
3250/3250 [==============================] - 63s 19ms/step - loss: 3.4100 - mean_squared_error: 3.4100 - val_loss: 3.6772 - val_mean_squared_error: 3.6772
Epoch 2/5
3250/3250 [==============================] - 63s 19ms/step - loss: 3.1895 - mean_squared_error: 3.1895 - val_loss: 3.5743 - val_mean_squared_error: 3.5743
Epoch 3/5
3250/3250 [==============================] - 64s 20ms/step - loss: 3.0716 - mean_squared_error: 3.0716 - val_loss: 3.4620 - val_mean_squared_error: 3.4620
Epoch 4/5
3250/3250 [==============================] - 63s 20ms/step - loss: 3.0144 - mean_squared_error: 3.0144 - val_loss: 3.4239 - val_mean_squared_error: 3.4239
Epoch 5/5
3250/3250 [==============================] - 64s 20ms/step - loss: 2.9832 - mean_squared_error: 2.9832 - val_loss: 3.4383 - val_mean_squared_error: 3.4383


In [299]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

813/813 [==============================] - 1s 1ms/step - loss: 3.4383 - mean_squared_error: 3.4383


[3.4382917881011963, 3.4382917881011963]

In [300]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

813/813 [==============================] - 1s 946us/step


array([[88.44615 ],
       [86.40053 ],
       [86.7319  ],
       [91.00393 ],
       [90.6269  ],
       [90.50606 ],
       [83.442535],
       [83.86459 ],
       [90.184685],
       [91.1009  ]], dtype=float32)

In [301]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([88., 86., 87., 91., 91., 91., 83., 84., 90., 91., 90., 97., 86.,
       91., 88., 92., 86., 90., 88., 88.], dtype=float32)>

In [302]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

{'MAE': 1.4372331012195592,
 'MSE': 3.5133305120609393,
 'R-squared': 0.6230455021301513}

In [303]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results, 
                                new_model_results=model_1_results)

Baseline MAE: 1.41, New MAE: 1.44, Difference: 0.03
Baseline MSE: 3.42, New MSE: 3.51, Difference: 0.09
Baseline R-squared: 0.63, New R-squared: 0.62, Difference: -0.01


# Model 2: LSTM

In [372]:


# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="linear")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 40, 128)
(None, 64)


In [375]:
# Compile model for regression
model_2.compile(loss="mean_squared_error",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mean_squared_error"])


In [376]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [377]:
# Fit the model
model_2_history = model_2.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=100,
                              validation_data=(val_sentences, val_labels),  callbacks=[early_stopping],
                              )

Epoch 1/100
3250/3250 [==============================] - 95s 29ms/step - loss: 884.0226 - mean_squared_error: 884.0226 - val_loss: 9.3303 - val_mean_squared_error: 9.3303
Epoch 2/100
3250/3250 [==============================] - 97s 30ms/step - loss: 9.2285 - mean_squared_error: 9.2285 - val_loss: 9.3255 - val_mean_squared_error: 9.3255
Epoch 3/100
3250/3250 [==============================] - 97s 30ms/step - loss: 8.7837 - mean_squared_error: 8.7837 - val_loss: 4.6252 - val_mean_squared_error: 4.6252
Epoch 4/100
3250/3250 [==============================] - 93s 29ms/step - loss: 3.6125 - mean_squared_error: 3.6125 - val_loss: 3.3056 - val_mean_squared_error: 3.3056
Epoch 5/100
3250/3250 [==============================] - 94s 29ms/step - loss: 2.9155 - mean_squared_error: 2.9155 - val_loss: 3.1543 - val_mean_squared_error: 3.1543
Epoch 6/100
3250/3250 [==============================] - 95s 29ms/step - loss: 2.6219 - mean_squared_error: 2.6219 - val_loss: 3.1082 - val_mean_squared_error: 3

In [378]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

813/813 [==============================] - 4s 5ms/step


((25993, 1),
 array([[87.91096 ],
        [88.67603 ],
        [86.357704],
        [91.01722 ],
        [89.61097 ],
        [89.647675],
        [82.598015],
        [83.27592 ],
        [91.06005 ],
        [92.27299 ]], dtype=float32))

In [379]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([88., 89., 86., 91., 90., 90., 83., 83., 91., 92.], dtype=float32)>

In [380]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'MAE': 1.3703304735890431,
 'MSE': 3.3854884007232715,
 'R-squared': 0.6367620194690347}

In [381]:
# Compare model 2 to baseline
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline MAE: 1.41, New MAE: 1.37, Difference: -0.04
Baseline MSE: 3.42, New MSE: 3.39, Difference: -0.04
Baseline R-squared: 0.63, New R-squared: 0.64, Difference: 0.00


# Model 3: Conv1D

In [333]:
# Test out the embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu") # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer
max_pool = layers.GlobalMaxPool1D() 
max_pool_output = max_pool(conv_1d_output) # get the most important features
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 40, 128]), TensorShape([1, 36, 32]), TensorShape([1, 32]))

In [334]:
# See the outputs of each layer
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 40, 128), dtype=float32, numpy=
 array([[[ 0.53918016, -0.6094435 ,  0.47952527, ...,  0.52343374,
           0.52661276, -0.56679463],
         [ 0.4566596 , -0.46478346,  0.5033103 , ...,  0.47044986,
           0.54187506, -0.5501663 ],
         [ 0.50307244, -0.57592446,  0.46362057, ...,  0.5673405 ,
           0.57735085, -0.5065696 ],
         ...,
         [ 0.49757418, -0.48028332,  0.46292648, ...,  0.5332629 ,
           0.47543415, -0.53812623],
         [ 0.49757418, -0.48028332,  0.46292648, ...,  0.5332629 ,
           0.47543415, -0.53812623],
         [ 0.49757418, -0.48028332,  0.46292648, ...,  0.5332629 ,
           0.47543415, -0.53812623]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 36, 32), dtype=float32, numpy=
 array([[[0.        , 0.512009  , 1.153581  , ..., 0.        ,
          0.49206328, 0.        ],
         [0.        , 0.5182718 , 1.2225544 , ..., 0.        ,
          0.59311503, 0.        ],
         [0.        , 0.5559645 , 1.24

In [335]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="linear")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_3.compile(loss="mean_squared_error",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mse"])

# Get a summary of our 1D convolution model
model_3.summary()


Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 40)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 40, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 36, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_55 (Dense)            (None, 1)              

In [336]:
# Fit the model
model_3_history = model_3.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              )

Epoch 1/5
3250/3250 [==============================] - 73s 22ms/step - loss: 214.9265 - mse: 214.9265 - val_loss: 6.2407 - val_mse: 6.2407
Epoch 2/5
3250/3250 [==============================] - 74s 23ms/step - loss: 5.3694 - mse: 5.3694 - val_loss: 4.7511 - val_mse: 4.7511
Epoch 3/5
3250/3250 [==============================] - 74s 23ms/step - loss: 4.1526 - mse: 4.1526 - val_loss: 4.2901 - val_mse: 4.2901
Epoch 4/5
3250/3250 [==============================] - 72s 22ms/step - loss: 3.4038 - mse: 3.4038 - val_loss: 4.2370 - val_mse: 4.2370
Epoch 5/5
3250/3250 [==============================] - 73s 23ms/step - loss: 2.7545 - mse: 2.7545 - val_loss: 3.8315 - val_mse: 3.8315


In [337]:
# Make predictions on the validation dataset
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10] # view the first 10

813/813 [==============================] - 2s 2ms/step


((25993, 1),
 array([[87.37346 ],
        [87.28283 ],
        [86.66328 ],
        [91.63394 ],
        [87.18992 ],
        [89.89209 ],
        [82.95627 ],
        [85.261795],
        [90.703476],
        [89.23794 ]], dtype=float32))

In [338]:
# Round out predictions and reduce to 1-dimensional array
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([87., 87., 87., 92., 87., 90., 83., 85., 91., 89.], dtype=float32)>

In [339]:
# Calculate LSTM model results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'MAE': 1.507290424345016,
 'MSE': 3.915015581117993,
 'R-squared': 0.5799476515327149}

# model 5 

# Using Pretrained Embeddings (transfer learning for NLP)

In [342]:
# Example of pretrained embedding with universal sentence encoder - https://tfhub.dev/google/universal-sentence-encoder/4
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878049 -0.01271502  0.03971542  0.08827761
  0.02680984  0.05589836 -0.0106873  -0.00597294  0.00639323 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180627  0.01512473 -0.05162927
  0.00991364 -0.06865346 -0.04209306  0.02678978  0.03011007  0.00321069
 -0.00337969 -0.04787355  0.02266721 -0.00985927 -0.04063614 -0.01292093
 -0.04666384  0.05630299 -0.03949255  0.00517685  0.02495828 -0.0701444
  0.02871508  0.04947681 -0.00633978 -0.08960194  0.0280712  -0.00808365
 -0.01360601  0.05998649 -0.10361788 -0.05195374  0.00232957 -0.0233253
 -0.03758107  0.03327728], shape=(50,), dtype=float32)


In [341]:
!pip install tensorflow_hub


     ------------------------------------ 100.6/100.6 kB 828.6 kB/s eta 0:00:00


In [343]:
# Each sentence has been encoded into a 512 dimension vector
embed_samples[0].shape

TensorShape([512])

In [344]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 

In [353]:


# Create model using the Sequential API
model_5 = tf.keras.Sequential([
  sentence_encoder_layer,  # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1 ,activation="linear")  # Remove the activation function for regression
], name="model_5_USE")

# Compile model
model_5.compile(loss=tf.keras.losses.MeanSquaredError(),  # Use Mean Squared Error loss for regression
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mse"])  # Use Mean Squared Error as the metric

model_5.summary()


Model: "model_5_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_62 (Dense)            (None, 64)                32832     
                                                                 
 dense_63 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [354]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [355]:
# Train a classifier on top of pretrained embeddings
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=100,
                              validation_data=(val_sentences, val_labels),  callbacks=[early_stopping],
                                                                    )

Epoch 1/100
3250/3250 [==============================] - 36s 11ms/step - loss: 383.4750 - mse: 383.4750 - val_loss: 32.1808 - val_mse: 32.1808
Epoch 2/100
3250/3250 [==============================] - 36s 11ms/step - loss: 20.7007 - mse: 20.7007 - val_loss: 12.2451 - val_mse: 12.2451
Epoch 3/100
3250/3250 [==============================] - 37s 11ms/step - loss: 9.2994 - mse: 9.2994 - val_loss: 7.4234 - val_mse: 7.4234
Epoch 4/100
3250/3250 [==============================] - 37s 11ms/step - loss: 6.6122 - mse: 6.6122 - val_loss: 5.9386 - val_mse: 5.9386
Epoch 5/100
3250/3250 [==============================] - 37s 11ms/step - loss: 5.6281 - mse: 5.6281 - val_loss: 5.4485 - val_mse: 5.4485
Epoch 6/100
3250/3250 [==============================] - 37s 11ms/step - loss: 5.1744 - mse: 5.1744 - val_loss: 6.1867 - val_mse: 6.1867
Epoch 7/100
3250/3250 [==============================] - 37s 11ms/step - loss: 4.9188 - mse: 4.9188 - val_loss: 4.8468 - val_mse: 4.8468
Epoch 8/100
3250/3250 [========

In [356]:
# Make predictions with USE TF Hub model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

813/813 [==============================] - 7s 8ms/step


array([[87.44683 ],
       [84.11611 ],
       [86.96642 ],
       [93.322395],
       [89.239174],
       [87.98503 ],
       [85.22346 ],
       [87.420876],
       [90.707375],
       [90.30432 ]], dtype=float32)

In [357]:
# Convert prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([87., 84., 87., 93., 89., 88., 85., 87., 91., 90.], dtype=float32)>

In [358]:
# Calculate model 6 performance metrics
model_5_results = calculate_results(val_labels, model_5_preds)
model_5_results

{'MAE': 1.6393259723771785,
 'MSE': 4.442465279113607,
 'R-squared': 0.523356182162841}